In [8]:
import zipfile as zf
import os
import pandas as pd
import numpy as np
import re

In [9]:
MAX = 8000
code_path = './Data'
test_path = './Test'

In [17]:
# zip files are put into folders (good/bad) based on team
path = './Data'
def createNParrays(path, maximum):
    code = []
    value = []
    for value_path in os.listdir(path):
        if os.path.isdir(path + '/' + value_path):
            for filename in os.listdir(path + '/' + value_path):
                if filename.endswith(".zip"):
                    processZip(path, value_path, filename, code, value)
    for i in range(len(code)):
        if len(code[i]) > maximum:
            code[i] = np.resize(code[i], (maximum))
        else:
            code[i] = np.pad(code[i], (0, maximum - len(code[i])), 'constant')
    X = np.array(code)
    print(X[0].shape)
    print(X.shape)
    np.save(path + '/code', X)
    np.save(path + '/values', np.array(value))
    return X, value
    
def processZip(path, value_path, filename, code, value):
    with zf.ZipFile(path + '/' + value_path + '/' + filename, 'r') as f:
        for name in f.namelist():
            if name.endswith('.java'):
                code.append(np.frombuffer(processFile(f, name), dtype=np.uint8))
                value.append(value_path)
        f.close()
        
def processFile(f, name):
    buffer = f.read(name)
    stringVersion = buffer.decode("utf-8")
    lines = stringVersion.splitlines()
    lines = [x for x in lines if not (x.startswith("import") or x.startswith("package")) ]
    newString = ''.join(lines)
    newString = removeComments(newString)
    return newString.encode("utf-8")

def removeComments(s):
    s = re.sub(re.compile("/\*.*?\*/",re.DOTALL ) ,"" ,s)
    s = re.sub(re.compile("//.*?\n" ) ,"" ,s)
    return s

In [18]:
X, value = createNParrays(code_path, MAX)
pd.get_dummies(value).sum(axis=0)

(8000,)
(163, 8000)


bad     128
good     35
dtype: int64

In [19]:
X, value = createNParrays(test_path, MAX)
pd.get_dummies(value).sum(axis=0)

(8000,)
(1643, 8000)


bad     915
good    728
dtype: int64